<a href="https://colab.research.google.com/github/sukhyun1017/pytorch-study/blob/main/pytorch_%ED%86%A0%EC%B9%98%ED%85%8D%EC%8A%A4%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
pip install torchtext

In [11]:
import urllib.request
import pandas as pd

In [15]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", filename="IMDb_Reviews.csv")

('IMDb_Reviews.csv', <http.client.HTTPMessage at 0x7f7ee6e3de10>)

In [16]:
df = pd.read_csv('IMDb_Reviews.csv', encoding='latin1')
df.head()

,review,sentiment
0,My family and I normally do not watch local mo...,1
1,"Believe it or not, this was at one time the wo...",0
2,"After some internet surfing, I found the ""Home...",0
3,One of the most unheralded great works of anim...,1
4,"It was the Sixties, and anyone with long hair ...",0


In [17]:
print('전체 샘플의 개수 : {}'.format(len(df)))

전체 샘플의 개수 : 50000


In [18]:
train_df = df[:25000]
test_df = df[25000:]

In [19]:
train_df.to_csv("train_data.csv", index=False)
test_df.to_csv("test_data.csv", index=False)

In [22]:
from torchtext.legacy import data # torchtext.data 임포트

# 필드 정의
TEXT = data.Field(sequential=True,    #시퀸스 데이터 여부
                  use_vocab=True,     #단어 집합을 만들 것인지 여부
                  tokenize=str.split, #어떤 토큰화 함수를 사용
                  lower=True,         # 전부 소문자화
                  batch_first=True,   #미니 배치 차원을 맨앞으로?
                  fix_length=20)      #최대허용 길이 여기에 맞춰서 패딩

LABEL = data.Field(sequential=False,
                   use_vocab=False,
                   batch_first=False,
                   is_target=True)    #레이블 데이터 여부
'''
batch_first = True -> 배치크기 x fix_length 형태로 미니배치가 만들어짐
batch_first = False -> fix_length x 배치크기 형태로 미니배치가 만들어짐

데이터셋 만들기

In [10]:
from torchtext.legacy.data import TabularDataset

In [23]:
train_data,test_data =TabularDataset.splits(path='.',train='train_data.csv',
                                            test='test_data.csv',format='csv',
                                            fields=[('text',TEXT),('label',LABEL)],
                                            skip_header=True) #데이터 첫째줄 무시

In [24]:
print('훈련 샘플의 개수 : {}'.format(len(train_data)))
print('테스트 샘플의 개수 : {}'.format(len(test_data)))

훈련 샘플의 개수 : 25000
테스트 샘플의 개수 : 25000


In [25]:
print(vars(train_data[0]))

{'text': ['my', 'family', 'and', 'i', 'normally', 'do', 'not', 'watch', 'local', 'movies', 'for', 'the', 'simple', 'reason', 'that', 'they', 'are', 'poorly', 'made,', 'they', 'lack', 'the', 'depth,', 'and', 'just', 'not', 'worth', 'our', 'time.<br', '/><br', '/>the', 'trailer', 'of', '"nasaan', 'ka', 'man"', 'caught', 'my', 'attention,', 'my', 'daughter', 'in', "law's", 'and', "daughter's", 'so', 'we', 'took', 'time', 'out', 'to', 'watch', 'it', 'this', 'afternoon.', 'the', 'movie', 'exceeded', 'our', 'expectations.', 'the', 'cinematography', 'was', 'very', 'good,', 'the', 'story', 'beautiful', 'and', 'the', 'acting', 'awesome.', 'jericho', 'rosales', 'was', 'really', 'very', 'good,', "so's", 'claudine', 'barretto.', 'the', 'fact', 'that', 'i', 'despised', 'diether', 'ocampo', 'proves', 'he', 'was', 'effective', 'at', 'his', 'role.', 'i', 'have', 'never', 'been', 'this', 'touched,', 'moved', 'and', 'affected', 'by', 'a', 'local', 'movie', 'before.', 'imagine', 'a', 'cynic', 'like', 'me

단어집합 만들기

In [27]:
TEXT.build_vocab(train_data,min_freq=10,max_size=10000)

In [28]:
print('단어 집합의 크기 : {}'.format(len(TEXT.vocab))) # <unk> 와 <pad> 자동추가

단어 집합의 크기 : 10002


In [ ]:
print(TEXT.vocab.stoi) # 결과가 너무 길어서 지움 .stoi로 각 단어가 어떤 정수에 할당됫는지 확인가능

데이터로더

In [31]:
from torchtext.legacy.data import Iterator #데이터 로더가 미니배치를 만들어준다

In [33]:
train_loader = Iterator(dataset=train_data, batch_size = 4)
test_loader = Iterator(dataset=test_data, batch_size = 4)

In [34]:
print('훈련 데이터의 미니 배치 수 : {}'.format(len(train_loader)))
print('테스트 데이터의 미니 배치 수 : {}'.format(len(test_loader)))

훈련 데이터의 미니 배치 수 : 6250
테스트 데이터의 미니 배치 수 : 6250


In [35]:
batch = next(iter(train_loader))

In [40]:
a = [1, 2, 3]
a_iter = iter(a)
next(a_iter)

1

In [36]:
print(type(batch))

<class 'torchtext.legacy.data.batch.Batch'>


In [37]:
print(batch.text)

tensor([[  10,   20,    7,    0,    4,    2,  252,    7,   49,    0,   20,    0,
           14,  950,    6,    0,   17,    2, 6132,    0],
        [  10,    7,   39,  714, 1280, 1850,   12,  197,  567,   47,   15, 2574,
           15, 4886,   70,   62,    0,    0,   12,   70],
        [  10,    7,    2, 1042,    6,    0,   13,    0, 4189,    3,  167,    5,
         2191, 7755,   18,  172,  500,  147,  755,   12],
        [2436,  107,    0,   18,  956,    0, 1514,  104,   40,  278,  228, 2255,
           48, 9457,   88,    7,   79,    0,    4, 6865]])


한국어 데이터

In [41]:
# Colab에 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 109 (delta 7), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (109/109), 1.27 MiB | 18.31 MiB/s, done.
Resolving deltas: 100% (46/46), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
     |████████████████████████████████| 19.4 MB 1.1 MB/s 
     |████████████████████████████████| 448 kB 55.0 MB/s 
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2022-03-13 16:19:02--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::6b17:d1f5, 2406:da00:ff00::22c0:3470, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Loc

In [42]:
import urllib.request
import pandas as pd

In [43]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x7f7eb62c2ed0>)

In [44]:
train_df = pd.read_table('ratings_train.txt')
test_df = pd.read_table('ratings_test.txt')

In [45]:
train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [46]:
print('훈련 데이터 샘플의 개수 : {}'.format(len(train_df)))
print('테스트 데이터 샘플의 개수 : {}'.format(len(test_df)))

훈련 데이터 샘플의 개수 : 150000
테스트 데이터 샘플의 개수 : 50000


In [50]:
from torchtext.legacy import data # torchtext.data 임포트
from konlpy.tag import Mecab

In [48]:
tokenizer = Mecab()

In [51]:
ID = data.Field(sequential = False,
                use_vocab = False) # 실제 사용은 하지 않을 예정

TEXT = data.Field(sequential=True,
                  use_vocab=True,
                  tokenize=tokenizer.morphs, # 토크나이저로는 Mecab 사용.
                  lower=True,
                  batch_first=True,
                  fix_length=20)

LABEL = data.Field(sequential=False,
                   use_vocab=False,
                   is_target=True)

데이터셋 만들기

In [52]:
from torchtext.legacy.data import TabularDataset

In [53]:
train_data, test_data = TabularDataset.splits(
        path='.', train='ratings_train.txt', test='ratings_test.txt', format='tsv',
        fields=[('id', ID), ('text', TEXT), ('label', LABEL)], skip_header=True)

In [54]:
print(vars(train_data[0]))

{'id': '9976970', 'text': ['아', '더', '빙', '.', '.', '진짜', '짜증', '나', '네요', '목소리'], 'label': '0'}


단어집합 만들기

In [55]:
TEXT.build_vocab(train_data, min_freq =10,max_size=10000)

In [56]:
print('단어 집합의 크기 : {}'.format(len(TEXT.vocab)))

단어 집합의 크기 : 10002


In [ ]:
print(TEXT.vocab.stoi) #.stoi 를 통해 단어집합 확인